In [9]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("C:/Users/admin/Desktop/NLP/Quora Sent analysis/input"))
dir = "C:/Users/admin/Desktop/NLP/Quora Sent analysis/input"
# Any results you write to the current directory are saved as output.

['sample_submission.csv', 'test.csv', 'train.csv']


In [8]:
import datetime
import nltk
import operator 
import re
from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [10]:
train_df = pd.read_csv(dir + "/train.csv")
train_df_len = train_df.shape[0]
print('train data length: {}'.format(train_df_len)) # 1306122
train_df.head()

train data length: 1306122


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [15]:
t0, t1 = len(train_df[train_df.target==0]), len(train_df[train_df.target==1])
t0_pct, t1_pct = t0 / train_df_len * 100, t1 / train_df_len * 100
print('target 0 vs 1 = {} vs {}, {:.2f}% vs {:.2f}%'.format(t0, t1, t0_pct, t1_pct))

target 0 vs 1 = 1225312 vs 80810, 93.81% vs 6.19%


In [16]:
test_df = pd.read_csv(dir + "/test.csv")
print('test data length: {}'.format(test_df.shape[0]))
test_df.head()

test data length: 375806


,qid,question_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...
3,000086e4b7e1c7146103,Who are entrepreneurs?
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?


In [17]:
sample_df = pd.read_csv(dir + '/sample_submission.csv')
print('sample submission length: {}'.format(sample_df.shape[0]))
sample_df.head()

sample submission length: 375806


,qid,prediction
0,0000163e3ea7c7a74cd7,0
1,00002bd4fb5d505b9161,0
2,00007756b4a147d2b0b3,0
3,000086e4b7e1c7146103,0
4,0000c4c3fbe8785a3090,0


Preprocessing 

In [18]:
# Contractions corrections
contraction_dict = {
    "ain't": "is not", "aren't": "are not", "can't": "cannot",
    "'cause": "because", "could've": "could have", "couldn't": "could not",
    "didn't": "did not", "doesn't": "does not", "don't": "do not",
    "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
    "he'd": "he would", "he'll": "he will", "he's": "he is",
    "how'd": "how did", "how'd'y": "how do you", "how'll": "how will",
    "how's": "how is", "I'd": "I would", "I'd've": "I would have",
    "I'll": "I will", "I'll've": "I will have", "I'm": "I am",
    "I've": "I have", "i'd": "i would", "i'd've": "i would have",
    "i'll": "i will",  "i'll've": "i will have", "i'm": "i am",
    "i've": "i have", "isn't": "is not", "it'd": "it would",
    "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have",
    "it's": "it is", "let's": "let us", "ma'am": "madam",
    "mayn't": "may not", "might've": "might have", "mightn't": "might not",
    "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
    "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have",
    "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
    "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",
    "she'd": "she would", "she'd've": "she would have", "she'll": "she will",
    "she'll've": "she will have", "she's": "she is", "should've": "should have",
    "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have",
    "so's": "so as", "this's": "this is", "that'd": "that would",
    "that'd've": "that would have", "that's": "that is", "there'd": "there would",
    "there'd've": "there would have", "there's": "there is", "here's": "here is",
    "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
    "they'll've": "they will have", "they're": "they are", "they've": "they have",
    "to've": "to have", "wasn't": "was not", "we'd": "we would",
    "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have",
    "we're": "we are", "we've": "we have", "weren't": "were not",
    "what'll": "what will", "what'll've": "what will have", "what're": "what are",
    "what's": "what is", "what've": "what have", "when's": "when is",
    "when've": "when have", "where'd": "where did", "where's": "where is",
    "where've": "where have", "who'll": "who will", "who'll've": "who will have",
    "who's": "who is", "who've": "who have", "why's": "why is",
    "why've": "why have", "will've": "will have", "won't": "will not",
    "won't've": "will not have", "would've": "would have", "wouldn't": "would not",
    "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
    "y'all'd've": "you all would have", "y'all're": "you all are", "y'all've": "you all have",
    "you'd": "you would", "you'd've": "you would have", "you'll": "you will",
    "you'll've": "you will have", "you're": "you are", "you've": "you have"
}

In [19]:
def clean_contractions(text, contraction_dict):
    specials = ["’", "‘", "´", "`"]
    for s in specials:
        text = text.replace(s, "'")
    text = ' '.join([contraction_dict[t] if t in contraction_dict else t for t in text.split(" ")])
    return text

In [20]:
# special characters
punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'

In [21]:
punct_dict = {
    "‘": "'",    "₹": "e",      "´": "'", "°": "",         "€": "e",
    "™": "tm",   "√": " sqrt ", "×": "x", "²": "2",        "—": "-",
    "–": "-",    "’": "'",      "_": "-", "`": "'",        '“': '"',
    '”': '"',    '“': '"',      "£": "e", '∞': 'infinity', 'θ': 'theta',
    '÷': '/',    'α': 'alpha',  '•': '.', 'à': 'a',        '−': '-',
    'β': 'beta', '∅': '',       '³': '3', 'π': 'pi'
}

In [22]:
def clean_special_chars(text, punct, punct_dict):
    for p in punct_dict:
        text = text.replace(p, punct_dict[p])
    
    for p in punct:
        text = text.replace(p, f' {p} ')
    
    specials = {'\u200b': ' ', '…': ' ... ', '\ufeff': '', 'करना': '', 'है': ''}  # Other special characters that I have to deal with in last
    for s in specials:
        text = text.replace(s, specials[s])
    
    return text

In [23]:
stopwords = nltk.corpus.stopwords.words('english')


In [24]:
def preprocess(df, contraction_dict, punct, punct_dict):
    texts = df.question_text
    processed_texts = texts.apply(lambda x: x.lower())
    processed_texts = processed_texts.apply(lambda x: clean_contractions(x, contraction_dict))
    processed_texts = processed_texts.apply(lambda x: clean_special_chars(x, punct, punct_dict))
    processed_texts = processed_texts.apply(lambda x: re.split('\W+', x))
    processed_texts = processed_texts.apply(lambda x: [token for token in x if token not in stopwords])
    df['processed_text'] = processed_texts

In [25]:
SAMPLE_ROWS_T0 = 639190
SAMPLE_ROWS_T1 = 80810
df_t0 = train_df[train_df.target==0].sample(SAMPLE_ROWS_T0)
df_t1 = train_df[train_df.target==1].sample(SAMPLE_ROWS_T1)

In [26]:
preprocess(df_t0, contraction_dict, punct, punct_dict)
df_t0.head()

,qid,question_text,target,processed_text
484879,5ef340417ffdee1f7fb9,What does it mean if my ex acts like he hates ...,0,"[mean, ex, acts, like, hates, always, looks, t..."
634472,7c417d73cf6e45ac9a47,What is it like working for the Canadian milit...,0,"[like, working, canadian, military, trans, per..."
678872,84f348126f2b253cb054,Why is it important to perform system analysis?,0,"[important, perform, system, analysis, ]"
860777,a8a436c049b231799c7b,When will my deposited money will be aviable i...,0,"[deposited, money, aviable, account, ]"
496088,6124e3ca86664e479bfd,Why does a pure tungsten filament emit a conti...,0,"[pure, tungsten, filament, emit, continuous, s..."


In [27]:
preprocess(df_t1, contraction_dict, punct, punct_dict)
df_t1.head()

,qid,question_text,target,processed_text
959197,bbedfe8afd5fef69da5c,Do you think a certain percentage of Americans...,1,"[think, certain, percentage, americans, always..."
274636,35bff8f8bde326bcd2ba,How did Xi Jinping made Apple sold out their i...,1,"[xi, jinping, made, apple, sold, icloud, china..."
147599,1cde4cc9bf5c04c4dd0f,If leftists are correct & Trump isn't as rich ...,1,"[leftists, correct, trump, rich, claims, pull, ]"
786660,9a1cac9321a23b9a5f05,Why has Brahmin bashing become so common?,1,"[brahmin, bashing, become, common, ]"
64128,0c9360bbd317712240ca,Why do foreigners claim that their Filipina wi...,1,"[foreigners, claim, filipina, wife, beautiful,..."


In [28]:
preprocess(test_df, contraction_dict, punct, punct_dict)
test_df.head()

,qid,question_text,processed_text
0,0000163e3ea7c7a74cd7,Why do so many women become so rude and arroga...,"[many, women, become, rude, arrogant, get, lit..."
1,00002bd4fb5d505b9161,When should I apply for RV college of engineer...,"[apply, rv, college, engineering, bms, college..."
2,00007756b4a147d2b0b3,What is it really like to be a nurse practitio...,"[really, like, nurse, practitioner, ]"
3,000086e4b7e1c7146103,Who are entrepreneurs?,"[entrepreneurs, ]"
4,0000c4c3fbe8785a3090,Is education really making good people nowadays?,"[education, really, making, good, people, nowa..."


Building vocabulary

In [29]:
def build_vocab(texts, vocab):
    for word in texts:
        vocab.add(word)

In [30]:
vocab = set()
df_t1.processed_text.apply(lambda x: build_vocab(x, vocab))
df_t0.processed_text.apply(lambda x: build_vocab(x, vocab))
test_df.processed_text.apply(lambda x: build_vocab(x, vocab))
print(len(vocab))

177342


In [31]:
vocab

{'',
 'gents',
 'scool',
 'logarithms',
 'overclockable',
 'kannads',
 'pitter',
 'angio',
 'longtime',
 'cartoon',
 'underexplored',
 'dalliance',
 'machinary',
 'mazandaranis',
 'giantess',
 'whyasansol',
 'plcs',
 'ardently',
 'declaring',
 'safex',
 'realistically',
 'yudkowsky',
 'itpin',
 'cannikin',
 'elastically',
 'hsync',
 'plr',
 'hoodwinked',
 'implying',
 'kakashi',
 'damate',
 'umbrical',
 'baguio',
 'bioweapons',
 'forenames',
 'avelox',
 'purdah',
 'starin',
 'fwrite',
 'enkidu',
 'collabarative',
 'umbilical',
 '5p',
 'dispensations',
 'novgorod',
 'quants',
 '500g',
 'agora',
 'xat',
 'defaulted',
 'insurable',
 'sizzler',
 'aircfaft',
 'andres',
 'loney',
 'keywest',
 'disrespectful',
 'katyn',
 'ba42',
 'sewall',
 'makut',
 'iriana',
 '13kgs',
 'mucinex',
 'wildlings',
 'manse',
 'cinematic',
 'maniacs',
 'shining',
 'winstrol',
 'amass',
 'g10m',
 'hapneed',
 'dimarco',
 'hwhbl',
 'lm337',
 'lucent',
 'lowlife',
 'indivudal',
 'lavoy',
 'alicyclic',
 'zauberberg',


Load embedding 

In [32]:
def load_embed(filename, vocab):
    word2vec = {}
    def get_coefs(word,*arr): 
        return word, np.asarray(arr, dtype='float32')
    f = open(filename, encoding='latin')
    for line in tqdm(f):
        word, coefs = get_coefs(*line.split(" "))
        if word in vocab:
            word2vec[word] = coefs
    f.close()
    return word2vec

In [40]:
paragram =  'C:/Users/admin/Desktop/NLP/Quora Sent analysis/paragram_300_sl999/paragram_300_sl999.txt'


In [41]:
word2vec = load_embed(paragram, vocab)


1703756it [02:28, 11456.21it/s]


In [42]:
len(word2vec), word2vec['add'].shape

(134653, (300,))

In [45]:
# see the train data closely
# min and max number of words in questions
lens_t0 = list(map(len, df_t0.processed_text))
lens_t1 = list(map(len, df_t1.processed_text))
lens_test = list(map(len, test_df.processed_text))
print('min and max words in pos questions: {}, {}'.format(min(lens_t0), max(lens_t0)))
print('min and max words in neg questions: {}, {}'.format(min(lens_t1), max(lens_t1)))
print('min and max words in test questions: {}, {}'.format(min(lens_test), max(lens_test)))

min and max words in pos questions: 1, 56
min and max words in neg questions: 1, 181
min and max words in test questions: 0, 239


In [48]:
def freq_stats(tag, counts, key, topk, total):
    most_freqs = sorted(counts, key=key, reverse=True)[:topk]
    freqs = [counts[freq] for freq in most_freqs]
    print('{}: best {} frequent word count: {}, '.format(tag, topk, most_freqs),
          'freqs: {}, '.format(freqs),
          'covers: {:.2f}%'.format(sum(freqs)/total*100))
    return max(most_freqs)

from collections import Counter
counts_t0 = Counter(lens_t0)
counts_t1 = Counter(lens_t1)
counts_test = Counter(lens_test)
#topk = 5 # vast majority of questions are covered, but may lose clues to classify correctly
topk = 20
max_t0 = freq_stats('pos', counts_t0, counts_t0.get, topk, SAMPLE_ROWS_T0)
max_t1 = freq_stats('neg', counts_t1, counts_t1.get, topk, SAMPLE_ROWS_T1)
max_test = freq_stats('test', counts_test, counts_test.get, topk, test_df.shape[0])

pos: best 20 frequent word count: [5, 6, 7, 4, 8, 9, 10, 3, 11, 12, 13, 14, 15, 16, 17, 2, 18, 19, 20, 21],  freqs: [105875, 103300, 82300, 80340, 61403, 44480, 32322, 28908, 23904, 17703, 13697, 10374, 7705, 5641, 4113, 3437, 3139, 2446, 1975, 1604],  covers: 99.29%
neg: best 20 frequent word count: [6, 7, 5, 8, 9, 10, 4, 11, 12, 13, 14, 15, 16, 17, 3, 18, 19, 20, 21, 22],  freqs: [8138, 7634, 7412, 7019, 6342, 5619, 5325, 5051, 4376, 3822, 3349, 2662, 2154, 1885, 1747, 1628, 1325, 1138, 1011, 855],  covers: 97.13%
test: best 20 frequent word count: [5, 6, 7, 4, 8, 9, 10, 3, 11, 12, 13, 14, 15, 16, 17, 18, 2, 19, 20, 21],  freqs: [61140, 59354, 47525, 45416, 35711, 26837, 19582, 16128, 14678, 11060, 8537, 6533, 4978, 3826, 2783, 2172, 1869, 1712, 1365, 1196],  covers: 99.09%


In [49]:
SEQ_LENGTH = max(max_t0, max_t1, max_test)
SEQ_LENGTH

22

In [51]:
def build_weights_matrix(word2vec):
    word_to_idx = {}
    weights_matrix = np.zeros((len(word2vec), 300))
    for i, (k, v) in enumerate(word2vec.items()):
        word_to_idx[k] = i
        weights_matrix[i] = v
    return word_to_idx, weights_matrix

In [52]:
word_to_idx, weight_matrix = build_weights_matrix(word2vec)

In [54]:
weight_matrix.shape

(134653, 300)

In [55]:
# the length of word vector: seq_length
def encode_question(word_to_idx, text, seq_length):
    encoded = []
    for word in text[:seq_length]:
        try:
            encoded.append(word_to_idx[word])
        except KeyError:
            # missing words in the table such typos or created words
            continue

    return np.array(encoded, dtype='int_')

In [56]:
# adds padding
def add_padding(numpy_array, seq_length):
    cur_length = numpy_array.shape[0]
    if cur_length < seq_length:
        padding = np.zeros((seq_length-cur_length, ), dtype='int_')
        return np.concatenate((padding, numpy_array))
    else:
        return numpy_array

In [57]:
def create_dataset(texts, label, word_to_idx, seq_length):
    texts_len = len(texts)
    y = np.array([label]*texts_len, dtype='float')
    X = []
    for i, text in enumerate(texts):
        text_array = encode_question(word_to_idx, text, seq_length)
        text_array = add_padding(text_array, seq_length)
        X.append(text_array)
    return np.array(X), y

In [58]:
# splits train data to train and validation
TEST_SIZE = 0.1
train_texts_t0, val_texts_t0 = train_test_split(df_t0.processed_text, test_size=TEST_SIZE)
train_texts_t1, val_texts_t1 = train_test_split(df_t1.processed_text, test_size=TEST_SIZE)

In [61]:
train_texts_t0

291945    [would, happen, quora, removes, anonymity, fea...
526817                [study, cfa, level, 1, mba, doable, ]
223523    [messages, passing, three, tiers, architecture...
930074    [much, income, get, karnataka, 1000views, indi...
789678    [helping, raise, awareness, crimes, help, decr...
                                ...                        
582669    [get, t3, supplements, without, prescription, ...
237908                   [girls, like, black, men, curls, ]
798790                           [stockpile, water, home, ]
429012                              [best, fiction, read, ]
614433    [get, previous, year, question, papers, master...
Name: processed_text, Length: 575271, dtype: object

In [62]:
train_X_t0, train_y_t0 = create_dataset(train_texts_t0, 0, word_to_idx, SEQ_LENGTH)

In [63]:
train_X_t1, train_y_t1 = create_dataset(train_texts_t1, 1, word_to_idx, SEQ_LENGTH)

In [65]:
train_X_t0.shape

(575271, 22)

In [66]:
train_X = np.concatenate((train_X_t0, train_X_t1))
train_y = np.concatenate((train_y_t0, train_y_t1))
print('shapes: train_X {}, train_y {}'.format(train_X.shape, train_y.shape))

shapes: train_X (648000, 22), train_y (648000,)


In [67]:
val_X_t0, val_y_t0 = create_dataset(val_texts_t0, 0, word_to_idx, SEQ_LENGTH)
val_X_t1, val_y_t1 = create_dataset(val_texts_t1, 1, word_to_idx, SEQ_LENGTH)

In [68]:
val_X = np.concatenate((val_X_t0, val_X_t1))
val_y = np.concatenate((val_y_t0, val_y_t1))
print('shapes: val_X {}, val_y {}'.format(val_X.shape, val_y.shape))

shapes: val_X (72000, 22), val_y (72000,)


Model pytorch

In [69]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset

print(torch.__version__)
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

1.4.0
cuda:0


In [70]:
# creates Tensor datasets
train_set = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
val_set = TensorDataset(torch.from_numpy(val_X), torch.from_numpy(val_y))

In [71]:
train_set

In [72]:
# creates dataloaders
# hyperparameter for data loading
#  - batch_size: size of one batch
BATCH_SIZE = 200

# make sure to SHUFFLE the training data
train_loader = DataLoader(train_set, shuffle=True, batch_size=BATCH_SIZE)
val_loader = DataLoader(val_set, shuffle=True, batch_size=BATCH_SIZE)

In [74]:
# Only LSTM (2 or 3 layers) model suffered an overfitting problem.
# To avoid the problem, GRU and average pooling layer were added.
# The overfitting got better, but still the problem exists.
class SentimentRNN(nn.Module):
    def __init__(self, weights, n_out, n_hidden, n_layers,
                 bidirectional=False, dropout=0.5, layer_dropout=0.3):
        super(SentimentRNN, self).__init__()

        self.n_out = n_out
        self.n_hidden = n_hidden
        self.n_layers = n_layers
        if bidirectional:
            self.direction = 2
        else:
            self.direction = 1

        num_embeddings, embedding_dim = weights.shape
        
        # embedding layer
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        self.embedding.weight.data.copy_(torch.from_numpy(weights))
        self.embedding.weight.requires_grad = False
        # for some reason from_pretrained doesn't work
        #self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(weights))
        
        # LSTM layer
        self.lstm = nn.LSTM(embedding_dim, n_hidden, n_layers,
                            batch_first=True, dropout=dropout,
                            bidirectional=bidirectional)
        
        # GRU layer
        self.gru = nn.GRU(embedding_dim, n_hidden, n_layers,
                          batch_first=True, dropout=dropout,
                          bidirectional=bidirectional)
        # Conv1d layer
        self.conv1d = nn.Conv1d(n_hidden*self.direction, (n_hidden*self.direction)//2, 1)
        # Average Pooling layer
        self.avp = nn.AvgPool1d(2)
        # Dropout layer
        self.dropout = nn.Dropout(layer_dropout)
        # Fully-conneted layer
        self.fc = nn.Linear((n_hidden*self.direction)//4*2, n_out)
        
        # Sigmoid activation layer
        self.sig = nn.Sigmoid()

    def forward(self, x, hidden):
        batch_size = x.size(0)
        seq_len = x.size(1)
        lstm_hidden, gru_hidden = hidden
        
        embeds = self.embedding(x)
        
        lstm_out, lstm_hidden = self.lstm(embeds, lstm_hidden)
        lstm_out = lstm_out.contiguous().view(-1, self.n_hidden*self.direction, seq_len)
        lstm_out = self.conv1d(lstm_out)
        lstm_out = lstm_out.contiguous().view(-1, seq_len, (self.n_hidden*self.direction)//2)
        lstm_out = self.avp(lstm_out)
        
        gru_out, gru_hidden = self.gru(embeds, gru_hidden)
        gru_out = gru_out.contiguous().view(-1, self.n_hidden*self.direction, seq_len)
        gru_out = self.conv1d(gru_out)
        gru_out = gru_out.contiguous().view(-1, seq_len, (self.n_hidden*self.direction)//2)
        gru_out = self.avp(gru_out)
        
        #out = (lstm_out + gru_out) / 2.0
        out = torch.cat((lstm_out, gru_out), 2)
        out = self.dropout(out)
        out = self.dropout(out)
        out = self.fc(out.float())
        sig_out = self.sig(out)

        sig_out = sig_out.view(batch_size, -1)
        sig_out = sig_out[:, -1] # get only last labels
        
        return sig_out, (lstm_hidden, gru_hidden)
    
    def init_hidden(self, batch_size, bidirectional=False):
        weight = next(self.parameters()).data
        # for LSTM (initial_hidden_state, initial_cell_state)
        lstm_hidden = (
            weight.new(self.n_layers*self.direction, batch_size, self.n_hidden).zero_().to(DEVICE),
            weight.new(self.n_layers*self.direction, batch_size, self.n_hidden).zero_().to(DEVICE)
        )
        # for GRU, initial_hidden_state
        gru_hidden = weight.new(self.n_layers*self.direction, batch_size, self.n_hidden).zero_().to(DEVICE)
        return lstm_hidden, gru_hidden

In [75]:
# hyperparameters
n_out = 1
#n_hidden = 512
n_hidden = 256
n_layers = 3

In [76]:
# instantiate the network
net = SentimentRNN(weight_matrix, n_out, n_hidden, n_layers, bidirectional=False).to(DEVICE)
net

SentimentRNN(
  (embedding): Embedding(134653, 300)
  (lstm): LSTM(300, 256, num_layers=3, batch_first=True, dropout=0.5)
  (gru): GRU(300, 256, num_layers=3, batch_first=True, dropout=0.5)
  (conv1d): Conv1d(256, 128, kernel_size=(1,), stride=(1,))
  (avp): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Linear(in_features=128, out_features=1, bias=True)
  (sig): Sigmoid()
)

Training 

In [77]:
# hyperparameters for training
#  - lr: learning rate
#  - epochs: number of epochs
lr = 0.00008
epochs = 10
clip = 5 # gradient clipping

In [78]:
# loss and optimizer functions
criterion = nn.BCELoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
# for now, scheduler is not used. (has a bigger step_size than epochs)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

In [79]:
def train(net, criterion, optimizer, train_loader, clip, epoch, epochs, gru=True):
    # paramters for printing
    counter = 0
    print_every = 500

    train_length = len(train_loader)
    
    # initialize hidden state
    hidden = net.init_hidden(BATCH_SIZE)
    
    train_losses = []

    # batch loop
    for inputs, labels in train_loader:
        counter += 1

        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        
        if gru:
            l_h, g_h = hidden
            # for LSTM
            l_h = tuple([each.data for each in l_h])
            # for GRU
            g_h = g_h.data
            hidden = (l_h, g_h)
        else:
            hidden = tuple([each.data for each in hidden])
        
        # zero accumulated gradients
        net.zero_grad()
        
        # get the output from the model
        outputs, hidden = net(inputs, hidden)

        # calcuate the loss and perform backprop
        loss = criterion(outputs.squeeze(), labels.float())
        loss.backward()

        # `clip_grad_norm` helps prevent the exploding gradient probelm in RNNs/ LSTMs
        nn.utils.clip_grad_norm_(net.parameters(), clip)
        optimizer.step()

        # loss stats
        if counter % print_every == 0:
            train_losses.append(loss.item())
            print("Epoch: {}/{}...".format(epoch+1, epochs),
                  "Step: {}...".format(counter),
                  "Train Loss: {:.6f}...".format(np.mean(train_losses)),
                  "Time: {}".format(datetime.datetime.now()))

In [80]:
# get validation loss
THRESHOLD = 0.6
def validate(net, criterion, val_loader, epoch, epochs, gru=True):
    hidden = net.init_hidden(BATCH_SIZE)
    val_losses = []
    with torch.no_grad():
        for inputs, labels in val_loader:

            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            
            if gru:
                val_l_h, val_g_h = hidden
                # for LSTM
                val_l_h = tuple([each.data for each in val_l_h])
                # for GRU
                val_g_h = val_g_h.data
                hidden = (val_l_h, val_g_h)
            else:
                hidden = tuple([each.data for each in hidden])

            outputs, hidden = net(inputs, hidden)
            val_loss = criterion(outputs.squeeze(), labels.float())
            val_losses.append(val_loss.item())

            acc = torch.eq(labels.float(), torch.round(outputs.squeeze())).sum().item()

        print("Epoch: {}/{}...".format(epoch+1, epochs),
              "Val Loss: {:.6f}".format(np.mean(val_losses)),
              "Val Acc: {}/{}".format(acc, BATCH_SIZE),
              "Time: {}".format(datetime.datetime.now()))

In [81]:
def run_train(net,
              criterion, optimizer, scheduler,
              epochs, train_loader, val_loader,
              clip, gru=True):
    for epoch in range(epochs):
        scheduler.step()
        train(net, criterion, optimizer, train_loader, clip, epoch, epochs, gru)
        validate(net, criterion, val_loader, epoch, epochs, gru)

In [82]:
run_train(net, criterion, optimizer, scheduler, epochs, train_loader, val_loader, clip, gru=True)

C:\Users\admin\Anaconda3\lib\site-packages\torch\optim\lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.IntTensor instead (while checking arguments for embedding)